In [509]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
import cufflinks as cf
import plotly.offline as py
import warnings
warnings.simplefilter("ignore")
from sklearn.model_selection import cross_val_score,KFold,GridSearchCV
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.linear_model import LinearRegression, Lasso,Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder
from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.metrics import r2_score,mean_squared_error,mean_squared_error

In [510]:
train_df=pd.read_csv(r"C:\Users\abhisv.CORP\3D Objects\Bigmart\train_v9rqX0R.csv")

In [511]:
train_df.shape

(8523, 12)

In [512]:
train_df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [513]:
train_df.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Outlet_Sales'],
      dtype='object')

In [514]:
print(len(train_df.columns))

12


In [515]:
cat=train_df.select_dtypes(include=['O'])
print(len(cat.columns))

7


In [516]:
numerical=train_df.select_dtypes(exclude=['O'])
print(len(numerical.columns))

5


In [517]:
train_df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [518]:
cond=(train_df['Item_Weight'].isna())

In [519]:
len(cond[cond==True])

1463

In [520]:
train_df['Item_Identifier'].nunique()

1559

In [521]:
train_df.loc[cond,'Item_Identifier'].nunique()

1142

In [522]:
train_df.loc[cond,'Item_Weight']
map1=dict(train_df.groupby('Item_Identifier')['Item_Weight'].median())
map1
#pd.pivot_table('Item_Weight',index='Item_Identifier',aggfunc=np.mean())
train_df.loc[cond,'Item_Weight']=train_df.loc[cond,'Item_Identifier'].apply(lambda x :map1[x])

In [523]:
train_df.isna().sum()

Item_Identifier                 0
Item_Weight                     4
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [524]:
cond1=(train_df['Item_Weight'].isna())
cond1[cond1==True]

927     True
1922    True
4187    True
5022    True
Name: Item_Weight, dtype: bool

In [525]:
train_df['Item_Weight'].fillna(train_df['Item_Weight'].median(),inplace=True)

In [526]:
train_df.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,8523.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.875314,0.066132,140.992782,1997.831867,2181.288914
std,4.645010,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.785000,0.026989,93.826500,1987.000000,834.247400
50%,12.650000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [527]:
train_df['Outlet_Size'].value_counts()

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

In [528]:
train_df['Outlet_Size']=train_df['Outlet_Size'].map({'Small':1,'Medium':2,'High':3})

In [529]:
null_loc=train_df[train_df['Outlet_Size'].isna()]['Outlet_Location_Type'].value_counts().index

In [530]:
null_loc

Index(['Tier 2', 'Tier 3'], dtype='object')

In [531]:
train_df[train_df['Outlet_Size'].isna()].groupby(['Outlet_Identifier'])['Outlet_Location_Type'].agg(pd.Series.mode)

Outlet_Identifier
OUT010    Tier 3
OUT017    Tier 2
OUT045    Tier 2
Name: Outlet_Location_Type, dtype: object

In [532]:
train_df.groupby(['Outlet_Location_Type'])['Outlet_Size'].agg(pd.Series.mode)

Outlet_Location_Type
Tier 1    1.0
Tier 2    1.0
Tier 3    2.0
Name: Outlet_Size, dtype: float64

In [533]:
train_df.groupby(['Outlet_Type'])['Outlet_Size'].agg(pd.Series.mode)

Outlet_Type
Grocery Store        1.0
Supermarket Type1    1.0
Supermarket Type2    2.0
Supermarket Type3    2.0
Name: Outlet_Size, dtype: float64

In [534]:
train_df[train_df['Outlet_Size'].isna()].groupby(['Outlet_Identifier'])['Outlet_Type'].agg(pd.Series.mode)

Outlet_Identifier
OUT010        Grocery Store
OUT017    Supermarket Type1
OUT045    Supermarket Type1
Name: Outlet_Type, dtype: object

In [535]:
cond=(train_df['Outlet_Size'].isna())
train_df.loc[cond & (train_df['Outlet_Identifier']=='OUT010') & (train_df['Outlet_Type']=='Grocery Store'),'Outlet_Size']=2
train_df.loc[cond & (train_df['Outlet_Identifier']=='OUT045') & (train_df['Outlet_Type']=='Supermarket Type1'),'Outlet_Size']=1
train_df.loc[cond & (train_df['Outlet_Identifier']=='OUT017') & (train_df['Outlet_Type']=='Supermarket Type1'),'Outlet_Size']=1

In [536]:
train_df['Outlet_Size'].isna().sum()

0

In [537]:
train_df['Outlet_Type'].value_counts()

Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
Name: Outlet_Type, dtype: int64

In [538]:
train_df['Outlet_Type']=train_df['Outlet_Type'].map({'Grocery Store':1,'Supermarket Type1':2,'Supermarket Type2':3,'Supermarket Type3':4})

In [539]:
train_df['Outlet_Location_Type'].value_counts()

Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: Outlet_Location_Type, dtype: int64

In [540]:
train_df['Outlet_Location_Type']=train_df['Outlet_Location_Type'].map({'Tier 1':1,'Tier 2':2,'Tier 3':3})

In [541]:
train_df.head(2)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,2.0,1,2,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,2.0,3,3,443.4228


In [542]:
train_df['Outlet_Identifier'].value_counts()

OUT027    935
OUT013    932
OUT049    930
OUT046    930
OUT035    930
OUT045    929
OUT018    928
OUT017    926
OUT010    555
OUT019    528
Name: Outlet_Identifier, dtype: int64

In [543]:
oi_getdummy=pd.get_dummies(train_df['Outlet_Identifier'],drop_first=True)

In [544]:
train_df=pd.concat([train_df,oi_getdummy],axis=1)

In [545]:
train_df.drop('Outlet_Identifier',axis=1,inplace=True)

In [546]:
train_df.head(3)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,OUT013,OUT017,OUT018,OUT019,OUT027,OUT035,OUT045,OUT046,OUT049
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,1999,2.0,1,2,3735.1380,0,0,0,0,0,0,0,0,1
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,2009,2.0,3,3,443.4228,0,0,1,0,0,0,0,0,0
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,1999,2.0,1,2,2097.2700,0,0,0,0,0,0,0,0,1


In [547]:
train_df['Item_Type'].value_counts()

Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64

In [548]:
it_type=pd.get_dummies(train_df['Item_Type'],drop_first=True)

In [549]:
train_df=pd.concat([train_df,it_type],axis=1)

In [550]:
train_df.drop('Item_Type',axis=1,inplace=True)

In [551]:
train_df.head(3)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,...,Fruits and Vegetables,Hard Drinks,Health and Hygiene,Household,Meat,Others,Seafood,Snack Foods,Soft Drinks,Starchy Foods
0,FDA15,9.30,Low Fat,0.016047,249.8092,1999,2.0,1,2,3735.1380,...,0,0,0,0,0,0,0,0,0,0
1,DRC01,5.92,Regular,0.019278,48.2692,2009,2.0,3,3,443.4228,...,0,0,0,0,0,0,0,0,1,0
2,FDN15,17.50,Low Fat,0.016760,141.6180,1999,2.0,1,2,2097.2700,...,0,0,0,0,1,0,0,0,0,0


In [552]:
train_df['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [553]:
ifc_dummy=pd.get_dummies(train_df['Item_Fat_Content'],drop_first=True)

In [554]:
train_df=pd.concat([train_df,ifc_dummy],axis=1)

In [555]:
train_df.drop('Item_Fat_Content',axis=1,inplace=True)

In [556]:
train_df.head(2)

,Item_Identifier,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,OUT013,...,Meat,Others,Seafood,Snack Foods,Soft Drinks,Starchy Foods,Low Fat,Regular,low fat,reg
0,FDA15,9.30,0.016047,249.8092,1999,2.0,1,2,3735.1380,0,...,0,0,0,0,0,0,1,0,0,0
1,DRC01,5.92,0.019278,48.2692,2009,2.0,3,3,443.4228,0,...,0,0,0,0,1,0,0,1,0,0


In [557]:
train_df.drop(['Item_Identifier'],axis=1,inplace=True)

In [558]:
Y=train_df['Item_Outlet_Sales']
Y=pd.DataFrame(Y)

In [559]:
train_df.drop('Item_Outlet_Sales',axis=1,inplace=True)

In [576]:
X=train_df

In [582]:
rfc=RandomForestRegressor()
rfc.fit(X,Y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [583]:
rfc.predict(X)

array([3567.15666,  443.4228 , 2586.03378, ..., 1275.14016, 1886.61088,
        941.37462])

In [585]:
accuracy=r2_score(Y,rfc.predict(X))
accuracy

0.9169531675668328

In [591]:
paramDT1={     "criterion": "mse",
              "min_samples_split": 250,#[i for i in range(3,500,10)],
              "max_depth": 10,# [i for i in range(2,,3)],
              "min_samples_leaf": 7,#[i for i in range(1,500,20)],
              "max_leaf_nodes": 130,#[i for i in range(2,100,20)],
              "max_features":'auto',# ['auto', 'sqrt', 'log2'],
              "n_estimators":100

              }

rfc= RandomForestRegressor(**paramDT1,random_state=45)

rfc.fit(X, Y)
ptrain=rfc.predict(X)